In [1]:
import pandas as pd

with open('Data/training.1600000.processed.noemoticon.csv','r',encoding='latin-1') as f:
    data = pd.read_csv(f)

data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [2]:
len(data.columns)

6

In [3]:
data.columns = ['0','1','2','3','4','5']

In [4]:
data['5'] = data['5'].apply(lambda x: x.split("."))

In [5]:
data['5'] = data['5'].apply(lambda x: [ i for i in x if len(i) >= 2])

In [6]:
data['5'][100]

['@mangaaa I hope they will increase the capacity fast, yesterday was such a pain',
 ' Got the fail whale +15 times in 2 hours']

In [23]:
import string
import concurrent
from tqdm.notebook import tqdm
import pickle
import psutil
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import nltk
from joblib import Parallel, delayed

class EnglishDataCleaning:
    def __init__(self,eng_data,save=False,name=None):
        self.save = save
        self.eng_data = eng_data
        self.name = name
    # this is normal cleaning method not using multiple cores
    def clean(self,data):
        result = []
        for i in tqdm(data):
            text = i
            stop_words = stopwords.words('english')
            final_str = []
            regular_ex = r'[^a-zA-Z0-9\s]'
            regular_ex_1 = r'[$\n]'
            text = text.lower()
            text = re.sub(regular_ex,'',text)
            text = re.sub(regular_ex_1,'',text)
            lemmatizer = WordNetLemmatizer()
            tokenization = nltk.word_tokenize(text)
            for w in tokenization:
                w = contractions.fix(w)
                if w not in stop_words:
                    final_str.append(lemmatizer.lemmatize(w))
            result.append(' '.join(final_str))
        try:
            return result
        finally:
            if self.save == True:
                with open(f'cleaned_eng_data_{self.name}','wb') as f:
                    pickle.dump(result,f)
                print("Data saved successfully...")
            
    
    # method using multiple cores
    def fast_cleaning(self,data):
        def init_worker(mps, fps, cut):
            memorizedPaths, filepaths, cutoff = mps, fps, cut
            DG = 1
        def clean(text):
            import contractions
            from nltk.stem import WordNetLemmatizer
            from nltk.corpus import stopwords
            import re
            import nltk

            stop_words = stopwords.words('english')
            final_str = []
            regular_ex = r'[^a-zA-Z0-9\s]'
            regular_ex_1 = r'[$\n]'
            text = text.lower()
            text = re.sub(regular_ex,'',text)
            text = re.sub(regular_ex_1,'',text)
            lemmatizer = WordNetLemmatizer()
            tokenization = nltk.word_tokenize(text)
            for w in tokenization:
                w = contractions.fix(w)
                if w not in stop_words:
                    final_str.append(lemmatizer.lemmatize(w))
            return ' '.join(final_str)
        try:
            result = Parallel(n_jobs=-1, prefer="processes", verbose=6)(
            delayed(clean)(i) for i in tqdm(data))
        finally:
            if self.save == True:
                with open(f'cleaned_eng_data_{self.name}','wb') as f:
                    pickle.dump(result,f)
                print("Data saved successfully...")

    def partial_hindi_cleaner(self,hin_data):
        print("Cleaning hindi data...")
        cleaned_hin_data = []
        for i in tqdm(range(len(hin_data))):
            text = hin_data[i]
            t = ""
            regular_ex_1 = r'[$\n]'
            regular_ex_2 = r'[a-zA-Z]'
            text = re.sub(regular_ex_1,'',text)
            text = re.sub(regular_ex_2,'',text)
            for c in text:
                if c not in string.punctuation:
                    t += c
            cleaned_hin_data.append(t)
        if self.save == True:
            with open(f'cleaned_hin_data_{self.name}','wb') as f:
                pickle.dump(cleaned_hin_data,f)
            print("Data saved successfully")
        return cleaned_hin_data
        

In [18]:
new_data = []
for i in data['5']:
    for j in i:
        new_data.append(j)

In [24]:
data_clean = EnglishDataCleaning(new_data,save=True,name="english_data")

In [ ]:
data_clean.fast_cleaning(new_data)

  0%|          | 0/2604061 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 107 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 2104 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 16568 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 34104 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done 54968 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 78904 tasks      | elapsed:   29.3s
[Parallel(n_jobs=-1)]: Done 106168 tasks      | elapsed:   35.4s
[Parallel(n_jobs=-1)]: Done 136504 tasks      | elapsed:   42.2s
[Parallel(n_jobs=-1)]: Done 170168 tasks      | elapsed:   49.8s
[Parallel(n_jobs=-1)]: Done 206904 tasks      | elapsed:   57.7s
[Parallel(n_jobs=-1)]: Done 246968 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 290104 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 336568 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Don

  0%|          | 0/2604061 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [16]:

for i in tqdm(range(len(new_data))):
    new_data[i] = clean_the_data(new_data[i])

  0%|          | 0/2604061 [00:00<?, ?it/s]


KeyboardInterrupt



In [11]:
cleaned_data.eng_data[100]

['@mangaaa I hope they will increase the capacity fast, yesterday was such a pain',
 ' Got the fail whale +15 times in 2 hours']